# What is Kubernetes?

Kubernetes coordinates a highly available cluster of computers that are connected to work as a single unit, and automates the distribution and scheduling of application containers across a cluster efficiently. Its laundry list of features includes:
 - Service discovery and load balancing
 - Storage orchestration
 - Automated rollouts and rollbacks
 - Automatic bin packing
 - Self-healing
 - Secret and configuration management

### Limitations
 - You must containerize applications by hand before they can be deployed with Kubernetes
 - Kubernetes does not offer automated deployment, so you must manually roll out updates to your cluster after updating your containerized application
 - Kubernetes does not provide application-level services, just the infrastructure to orchestrate complex combinations of containerized applications
 - Interactions with the Kubernetes system are carried out on the command line, rather than with a GUI, so effective use of the Kubernetes infrastructure requires knowledge of the available commands and their options

# How does Kubernetes work?
 - **Pods:** A group of one or more containers, with shared storage and network resources, and a specification for how to run the containers.
 - **Services:** An abstraction which defines a logical set of Pods and a policy by which to access them.
 - **Ingress controller:** An API object that manages external access to the services in a cluster, typically with the HTTP, or HTTPS, protocol.
 - **Volumes:** A directory, possibly containing some data, which is accessible to the containers in a Pod.
 - **Namespaces:** Provide scopes for names and are primarily for use in environments with many users spread across multiple teams or projects.
 - **Secrets:** Objects that contain a small amount of sensitive data (e.g., passwords, tokens, or keys).

# Let's Get Started!
We will be using a simple web app, created with [Flask](https://flask.palletsprojects.com/en/2.0.x/), to demonstrate the core functionality of Kubernetes. The `app/main.py` script contains code which configures and initializes a local Flask development server. Its contents are included below:

In [1]:
import socket
from flask import Flask
app = Flask(__name__)


@app.route("/")
def hello():
    return f"Hello from Python! (IP: {socket.gethostbyname(socket.gethostname())})"


if __name__ == "__main__":
    app.run(host='0.0.0.0', port=80)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.104:5000/ (Press CTRL+C to quit)


Kubernetes allows you to deploy containerized applications to a cluster without tying them specifically to individual machines. A Kubernetes cluster is composed of two types of resources. First, the **Control Plane** handles the management of the cluster, coordinating all activities therein, such as: scheduling applications, maintaining applications' desired state, scaling applications, and rolling out application updates. Second, **Nodes** are physical computers or virtual machines that serve as worker machines in a cluster, running application instances. The following diagram illustrates the architecture of a Kubernetes cluster:

<div style="width: 100%;"><img src="img/module_01_cluster.svg"/></div>

The individual nodes in a cluster communicate with the control plane using the [Kubernetes API](https://kubernetes.io/docs/concepts/overview/kubernetes-api/). If you followed the setup steps for this workshop (located in `README.md`), and did not encounter any issues along the way, you should already have a cluster running locally. Run the following in your terminal/command prompt to view information about active clusters:

In [ ]:
kubectl cluster-info

Then, you can use the following command to view the names, statuses, roles, ages, and versions of individual Nodes:

In [ ]:
kubectl get nodes

Once you have a Kubernetes cluster running, you can begin to deploy your containerized application by creating a **Deployment** configuration. This Deployment tells Kubernetes how to create and update instances of your application. Once a Deployment has been created, the control plane schedules the application instances included in that Deployment to run on individual Nodes in the cluster. A **Deployment Controller** continuously monitors the created application instances, replacing instances that go down or are deleted with an instance running on another Node in the cluster. This provides a self-healing mechanism to address machine failure or maintenance. Now that we have a better idea of what goes on inside individual Nodes in a cluster, let's take a look at an updated cluster diagram.

<div style="width: 100%;"><img src="img/module_02_cluster.svg"/></div>

Before you deploy your Flask application, you need to containerize it. To do so, you must first navigate to the `app/` directory within this workshop folder in a terminal/command prompt window. Starting in the top-level repository directory `.../2022-spring-workshops`, you can use the following command to do so:

In [ ]:
cd "01 - Kubernetes"/app

Then, you need to create an image of the application (think of this as a snapshot of the app's current state), which can be done with the `docker build` command as follows:

In [ ]:
docker build -f Dockerfile -t flask-app:1.0 .

The `-f` option allows you to specify the filename of the Dockerfile to build the container image from. Using `-t` enables you to name the image and optional tag it with a version in the form `name:tag`. Here, we name the image `flask-app` and tag it as version `1.0`. The `.` at the end of the command specifies the context from which to build the image from, the current working directory in this case. Once your image has been built, you can run the following command to list your Docker images:

In [ ]:
docker image ls

If you see an image with a repository of `flask-app` and a tag of `1.0`, you successfully containerized your Flask application and are ready to deploy it locally with Kubernetes! By default, Kubernetes will attempt to pull Docker images from *the public Docker registry* when creating a Deployment. To avoid this behavior, you need to run a local Docker registry to host your container images. You can start a local Docker registry with the following command:

In [ ]:
docker run -d -p 5000:5000 --restart=always --name registry registry:2

Next, you need to tag the image you created so that you can push it to the local Docker registry you created. This can be achieved with the following command:

In [ ]:
docker tag flask-app:1.0 localhost:5000/flask-app:1.0

Then, push your container image to your local Docker registry with the following command:

In [ ]:
docker push localhost:5000/flask-app:1.0

Finally, you can create a new Deployment from the containerized Flask application by running the following command:

In [ ]:
kubectl create deployment flask-app --image=localhost:5000/flask-app:1.0

Running the `create deployment` command causes the following actions to be performed:
- searching for a suitable node where an instance of the application could be run
- scheduling the application to run on that Node
- configuring the cluster to reschedule the instance on a new Node when needed

You can then list your Deployments with the following command:

In [ ]:
kubectl get deployments

The `READY` column indicates the number of instances of your app that are running, followed by the number of deployments you created. Your output should indicate that there is 1 instance of your app running on the 1 Deployment you created. Currently, your deployed application can only be accessed from inside Kubernetes/kubectl. Later on you will learn how to set up a **Service** to expose your application, but for now you will use a simple proxy server provided by `kubectl`. *Open a new terminal/command prompt window* and run the following command:

In [ ]:
kubectl proxy

You can then view all the APIs hosted through the proxy endpoint with the following command, or by copy-pasting the URL into your web browser:

In [ ]:
curl http://localhost:8001

Run `kubectl get pods` and note the name of the Pod running your `flask-app`. You can then access the pod directly through the API by running the following command, or by copy-pasting the URL into your web browser (make sure to replace `POD-NAME` with the name of the Pod you noted previously): 

In [ ]:
curl http://localhost:8001/api/v1/namespaces/default/pods/POD-NAME/

What is a Kubernetes **Pod**, though? Thus far, you may have a general idea of what Pods are, but we will now formally define them. Pods represent groups of one or more application containers (e.g., Docker containers), and some shared resources for those containers, including: shared storage (i.e., **Volumes**), networking (i.e., a unique cluster IP address), and information about how to run each individual container (e.g., container image version or specific ports to use). They are the atomic unit on the Kubernetes platform. When you create a Deployment, that Deployment will create Pod(s) with containers inside of them. The following infographic illustrates a few examples of Kubernetes Pods:

<div style="width: 100%;"><img src="img/module_03_pods.svg"/></div>

A **Pod** runs on a **Node**. A **Node** can have multiple **Pods**, and the **Control Plane** automatically handles scheduling the **Pods** across the **Nodes** in the cluster. Every Kubernetes Node runs at least:
- Kubelet, a process responsible for communication between the Kubernetes control plane and the Node; it manages the Pods and the containers running on a machine.
- A container runtime (like Docker) responsible for pulling the container image from a registry, unpacking the container, and running the application.

The architecture of a Kubernetes Node is illustrated by the following image:

<div style="width: 100%;"><img src="img/module_03_nodes.svg"/></div>

Up to this point, you have used a few of the commands implemented by the `kubectl` command-line interface. Some of the most well known `kubectl` operations are:
- **`kubectl get`** - list resources
- **`kubectl describe`** - show detailed information about a resource
- **`kubectl logs`** - print the logs from a container in a pod
- **`kubectl exec`** - execute a command on a container in a pod

You will now use some of the above commands to explore your existing application Deployment. Use the following command to see extensive details about the Pods in your cluster:

In [ ]:
kubectl describe pods

The `describe` command can be used to get detailed information about most of the Kubernetes primitives, such as Nodes, Pods, and Deployments. Similar to before, note the name of your Pod from the output of the `describe` command for use shortly. You can then access the Pod's API directly by running the following command, or by copy-pasting the URL into your web browser (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
curl http://localhost:8001/api/v1/namespaces/default/pods/POD-NAME/proxy/

Since your Flask application is incredibly simple, you should be greeted by a straightforward response from the Pod which contains the message: `Hello from Python!`. You can then look at the logs for the Pod you just sent an HTTP request to with the following command (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
kubectl logs POD-NAME

Anything that the application instance running in the Pod would regularly print to `STDOUT` (the standard output stream) is recorded in the logs for the container within the Pod. For your application, you should see some messages from when the Flask development server started up as well as one or more lines describing HTTP requests made to the server. Whether you used `curl` to send a request, or you navigated to the URL in your web browser, you should see at least one HTTP GET request listed in the output from `kubectl logs`.

Another useful command to know is `kubectl exec`, which allows you to execute commands directly on the container running in a Pod. The following command lists the environment variables for the container in the Pod that you have been working with thus far (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
kubectl exec POD-NAME -- env

To further illustrate the utility of `kubectl exec`, run the following command to start an interactive shell session in the Pod's container (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
kubectl exec -ti POD-NAME -- /bin/sh

Now that you are able to execute shell commands from within the container, run the following command to view the contents of the `main.py` script containing the code for the Flask application:

In [ ]:
cat main.py

To close your connection to the Pod's container, simply type `exit` and press your enter key.

Next, you will learn about Kubernetes **Services** and **Labels**. A Service routes traffic across a set of Pods, allowing Pods to die and replicate in Kubernetes without impacting your application. Labels are key/value pairs attached to objects and can be used in any number of ways:
- Designate objects for development, test, and production
- Embed version tags
- Classify an object using tags

Labels can be attached to objects at creation time or later on, and can be modified at any time. The following image illustrates the architecture of a cluster which utilizes services and labels:

<div style="width: 100%;"><img src="img/module_04_labels.svg"/></div>

To create a new Service and expose it to external traffic, we'll use the `kubectl expose` command, as follows:

In [ ]:
kubectl expose deployment/flask-app --type=LoadBalancer --port=8080 --target-port=80

Notice here that you expose port 8080 externally (with the `--port` option), and direct traffic to port 80 of the container itself (with the `--target-port` option). In essence, you can now access the container via port 8080 of the external IP address of the newly created Service. What is the external IP address of the new Service, though? You can use the `kubectl get` command to view some details about active Services in your cluster:

In [ ]:
kubectl get services

You should see two Services listed: one named `kubernetes` (which defines the cluster's IP address), and one named `flask-app` which you just created. Note the `EXTERNAL-IP` of the `flask-app` Service you created, which will likely be `localhost`. Next, use the following command to display detailed information about the newly created Service:

In [ ]:
kubectl describe services/flask-app

The Service you created allows you to access the Flask application directly, without routing your request through the Kubernetes proxy server that you should still have running. To do so, run the following command (make sure to replace `EXTERNAL-IP` with the `EXTERNAL-IP` listed in the `kubectl get services` output from before):

In [ ]:
curl EXTERNAL-IP:8080

Upon running this command, you should see the same, simple message returned from the Flask application as before. You can now close the separate terminal/command prompt window which you had running `kubectl proxy`.

When you created your Deployment, a label was automatically created for your Pod. Run the following command to view the name of the label created:

In [ ]:
kubectl describe deployment

The label for your Pod should be `app=flask-app`. Now, you can run the `kubectl get` command with the `-l` option which allows you to only get Pods with a label matching the one specified. Run the following:

In [ ]:
kubectl get pods -l app=flask-app

You likely only have one Pod in your cluster, so the output of the above command will likely be identical to the output of `kubectl get pods` without the `-l` option, but the ability to filter by label would be extremely useful in the case of multiple Pods/Deployments/etc. This option is perhaps better illustrated in such a case. Recall that the output of `kubectl get services` contains entries for a default Service created by Kubernetes, as well as the LoadBalancer Service that you created. Now try adding the `-l` option to `kubectl get services` as follows:

In [ ]:
kubectl get services -l app=flask-app

There should now only be the single Service that you created listed in the output. You can also give Labels to objects in your cluster manually, of course! Look back at the output from `kubectl get pods -l app=flask-app` and note the `NAME` of your Pod. Then, run the following command to add a new label to the Pod (make sure to replace POD-NAME with the name of the Pod):

In [ ]:
kubectl label pods POD-NAME version=v1

Then, use the following command to view the Pod's labels, among other information (make sure to replace POD-NAME with the name of the Pod you noted previously):

In [ ]:
kubectl describe pods POD-NAME

You should now see that the Pod has 3 labels: `app`, `pod-template-hash`, and `version`. Like before, we can filter the results of `kubectl get pods` with our newly created label with the following command:

In [ ]:
kubectl get pods -l version=v1

Now that you understand deploying and managing an application, you will next learn about scaling an application. If traffic was to drastically increase to your basic Flask website, you would need to scale the application to keep up with user demand. **Scaling** is accomplished by changing the number of replicas in a Deployment. The following two images illustrate the act of Scaling an application up.

<div style="width: 100%; display: flex;"><img src="img/module_05_scaling1.svg" style="width: 50%;"/><img src="img/module_05_scaling2.svg" style="width: 50%;"/></div>

The left image is akin to your current Kubernetes cluster, with a single Replica of your Deployment and a Service which exposes the Deployment externally. On the right, notice that there is still a single Service routing traffic to each of the now 4 Replicas of your Deployment. Kubernetes supports both manual and autoscaling of Deployments, but you will focus on manual Scaling in this workshop.

When you create a Deployment in Kubernetes, a **ReplicaSet** is automatically created which maintains a specific number of Pods running a Deployment. In your case, you should have a single ReplicaSet, which is maintaining a single Pod, in your cluster. You can list ReplicaSets in your cluster with the following command:

In [ ]:
kubectl get rs

The output here includes the following information of interest:
- `DESIRED`: the desired number of replicas of your application
- `CURRENT`: the current number of replicas that are currently running your application

As mentioned previously, your ReplicaSet should have 1 `DESIRED` Replica and 1 `CURRENT` Replica. Now, to scale your Deployment up, you can use the following command:

In [ ]:
kubectl scale deployments/flask-app --replicas=4

If you then list your Deployments again, you should see a value of `4/4` in the `READY` column, which indicates that there are 4 instances of the application running. Recall that you can use the following command to list your Deployments:

In [ ]:
kubectl get deployments

If you now inspect the Pods in your cluster, you should notice a few changes. Recall that you can list the Pods in your cluster with the following command (the `-o wide` option simply includes additional information in the output):

In [ ]:
kubectl get pods -o wide

There should now be 4 Pods running in your cluster, each with a unique IP address and a partially-randomized name. Notice also that 1 of your 4 Pods is much older than the other 3 (see the `AGE` column of the output). This means that Kubernetes did not dispose of the original Pod running your application, but simply added 3 new Pods to meet the requirement of having 4 Replicas of your Deployment.

As mentioned previously, you can use the `kubectl describe` command, to view noteworthy events related to various objects in your Kubernetes cluster. Use the following command to view events related to your Deployment:

In [1]:
kubectl describe deployments/flask-app

SyntaxError: invalid syntax (1746860440.py, line 1)

In the `EVENTS` section at the end of the output, observe that a `deployment-controller` initiated an event with a `REASON` of `ScalingReplicaSet`. The message for this event indicates that the ReplicaSet for your Deployment was scaled up to have 4 Replicas. Now, try using `curl` to send multiple requests to the same URL that you did previously. In my case, it was `localhost:8080`, so I should run the following command multiple times: 

In [ ]:
curl localhost:8080

The Flask app includes the local IP address of the machine running the application. Since we now have 4 separate Pods running the app, you should notice that the IP addresses included in the output from `curl` should vary from request to request. If this is the case, then your LoadBalancer Service is properly routing requests to different Pods. In this case, however, you do not need 4 Replicas of your application, because your cluster is not configured/hosted to recieve requests from outside your computer. As a result, you should scale your Deployment back down a bit, say to 2 Replicas, so you aren't using more resources than necessary to run your application. This can be achieved with the following command:

In [ ]:
kubectl scale deployments/flask-app --replicas=2

If you wanted to inspect your Deployments or Pods again to ensure that your Scaling was successful, you can run the following two commands that you've used previously:

In [ ]:
kubectl get deployments

In [ ]:
kubectl get pods -o wide

The final Kubernetes topic left to cover is updating your application. In Kubernetes, you can deploy new versions of your application with **Rolling Updates**, which allow a Deployment's update to take place with zero downtime by incrementally updating Pod instances with new ones. It is important to note that multiple instances of your application should be running in order to perform a rolling update without affecting application availability. If instead you only have one application instance, your LoadBalancer will be unable to route requests to the single Pod in your cluster as it is updating. The following series of images illustrates a high-level overview of how Rolling Updates affect the objects in your Kubernetes cluster.

<div style="width: 100%; display: flex;"><img src="img/module_06_rollingupdates1.svg" style="width: 25%;"/><img src="img/module_06_rollingupdates2.svg" style="width: 25%;"/><img src="img/module_06_rollingupdates3.svg" style="width: 25%;"/><img src="img/module_06_rollingupdates4.svg" style="width: 25%;"/></div>

Rolling Updates allow the following actions:
- Promote an application from one environment to another (via container image updates)
- Rollback to previous versions
- Continuous Integration and Continuous Delivery of applications with zero downtime

Before performing an update for your Deployment, you will need to update your Flask application! First, change the message returned by the `hello()` function in the `app/main.py` script. Then, use the following commands to containerize the new version of your application and push it to your locally-running Docker registry, respectively:

In [ ]:
# NOTE: Make sure your current working directory is the `01 - Kubernetes/app` folder before running this command
docker build -f Dockerfile -t localhost:5000/flask-app:1.1 .

In [ ]:
docker push localhost:5000/flask-app:1.1

Then, all you need to do to roll out your updated Flask application is to run the following command:

In [ ]:
kubectl set image deployments/flask-app flask-app=localhost:5000/flask-app:1.1

Like before, use `curl` to send a request to your application now that you've rolled out a new update (see example below). The output from `curl` should contain the new message that you wrote in `app/main.py`!

In [ ]:
curl localhost:8080

If you had a much larger Deployment (i.e., one that involved a large number of Replicas), then it may take some time to complete a Rolling Update. To check the status of an update rollout, you can use the following command:

In [ ]:
kubectl rollout status deployments/flask-app

In your case, you should see a message like `deployment "flask-app" successfully rolled out` as it does not take much time to replace/update 2 Pods. What if you roll out an update for your application, but you need to revert the Deployment back to the previous update? Thankfully, this can be achieved easily and effectively with the following command:

In [ ]:
kubectl rollout undo deployments/flask-app

The `kubectl rollout undo` command restores a Deployment to its last known state. Since updates are versioned, you can revert to any previously known state of a Deployment. To verify that rolling back the update worked, use `curl` to send a request to your application once more, like so:

In [ ]:
curl localhost:8080